# <center> Basic Data Preparation </center>

<center> <img src="https://i.ytimg.com/vi/BiPzz1xZ0XI/maxresdefault.jpg" width=800 height=600 /> </center>

## Basic ToolBox  - Preparação dos Dados II

A prparação é uma etapa de pré-processamento dos dados, antes de gerar o modelo definitivo, esta é justamente a etapa para preparar os dados antes de inseri los no modelo
 

In [5]:
# libs
#!pip install category-encoders
#!pip install  feature-engine
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder
from category_encoders import TargetEncoder
#from feature_engine.categorical_encoders import CountFrequencyCategoricalEncoder
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

## Variáveis Qualitativas (Categóricas)

Variáveis numéricas são mais faceis de se lhe dar, pois, não apresentam muitos problemas em produção. Quando falamos em **variáveis qualitativas** aí sim, possuímos problemas em potencial, para os quais precisamos ter uma rota de fuga.

Os 2 principais problemas são:
- Novos níveis nas variáveis
- Níveis raros

#### Carregando dados

In [6]:
df_titanic = pd.read_csv('titanic_prep.csv')

In [7]:
#df_titanic = pd.read_csv('https://raw.githubusercontent.com/abnr/ml-data/main/titanic_reduced.csv')
df_titanic.head(3)

,Embarked,Age,Sex,Pclass,Survived
0,S,22.0,male,3rd,0
1,C,38.0,female,1st,1
2,S,26.0,female,3rd,1


O DataFrame acima possui uma **variavel qualitativa nominal** "Sex" e duas **variavel qualitativa ordinal** "Pclass" e "Embarked"

#### Limpando os Dados

In [8]:
# Checando a existencia de valores ausentes, nulos
df_titanic.isna().sum()

Embarked      2
Age         177
Sex           0
Pclass        0
Survived      0
dtype: int64

In [9]:
# removendo dados ausentes, nulos
df_titanic.dropna(inplace=True)

In [10]:
# Confirmando a remoção de valores ausentes, nulos.
df_titanic.isna().sum()

Embarked    0
Age         0
Sex         0
Pclass      0
Survived    0
dtype: int64

#### Segregando as Variaveis 

In [11]:
# Variaveis Independentes
X_titanic = df_titanic.drop('Survived', axis=1)

# Variavel Dependente
Y_titanic = df_titanic['Survived']

#### Dividindo Em Treino e Teste

In [12]:
X_train_titanic, X_test_titanic, y_train_titanic, y_test_titanic = train_test_split(
    X_titanic, 
    Y_titanic,
    train_size=0.85,
    random_state=123)

## ``One-Hot Encoder``

O **One-Hot Encoding** é uma técnica de **pré-processamento de dados** que **converte variáveis categóricas** em **binários**. Em outras palavras, ele **cria uma nova coluna** para **cada valor único** presente na **variável categórica** e **atribui o valor 1** à coluna correspondente ao **valor presente** e **0 no resto das colunas**. O **one-hot enconding** e funciona melhor para as **variáveis em escala nominal**. Esse processo cria uma variável nova para cada variável string.

**One Hot encoding** é utilizado para a **preservação da informação dos valores categóricos originais**, ao criar **novas colunas** para **cada valor único**. Isso permite que os algoritmos de machine learning considerem essa informação durante o treinamento e tomada de decisões.

De maneira resuminda, o **One Hot encoding**, é uma **transformação** que fazemos nos dados para representarmos uma **variável categórica** de forma **binária**.

#### Vantagens:

* Preserva a distância entre as categorias, pois cria colunas binárias distintas para cada categoria.

* Adequado para variáveis categóricas sem ordem ou quando a ordem não é relevante.

* Pode ser facilmente interpretado por algoritmos de aprendizado de máquina.

#### Desvantagens:

* Pode levar a um aumento significativo no número de variáveis, especialmente se a variável categórica tiver muitas categorias.
* Pode causar o problema de multicolinearidade, onde as colunas binárias estão altamente correlacionadas.

<center> <img src="https://i.imgur.com/TW5m0aJ.png" height=400 width=600/> </center>

#### sklearn.preprocessing.``OneHotEncoder``

* Codifique recursos categóricos como uma matriz numérica one-hot. A entrada para este transformador deve ser uma matriz de números inteiros ou strings, denotando os valores assumidos por recursos categóricos (discretos). Os recursos são codificados usando um esquema de codificação one-hot (também conhecido como 'one-of-K' ou 'fictício'). Isso cria uma coluna binária para cada categoria e retorna uma matriz esparsa ou matriz densa (dependendo do arâmetro sparse_output). Por padrão, o codificador deriva as categorias com base nos valores exclusivos de cada recurso. Como alternativa, você também pode especificar categories manualmente. Essa codificação é necessária para alimentar dados categóricos para muitos estimadores do scikit-learn, principalmente modelos lineares e SVMs com os kernels padrão. 

**Nota**: Uma codificação one-hot de rótulos y deve usar um LabelBinarizer.Um dos principais motivos para utilizarmos o One-Hot Encoder no lugar do pd.get_dummies() é sua integração com a classe e com Pipelines do scikit-learn.

#### Preparando os Dados

In [13]:
# Atribuindo a uma variavel python as variveis qualitativas
cat_cols = ['Embarked', 'Pclass', 'Sex']

In [14]:
# Atribuindo a uma variavel python todas os dados das variaveis qualitativas
X_train_titanic_ = X_train_titanic.loc[:,cat_cols]

In [15]:
# Checando valores ausentes
X_train_titanic_.isna().sum()
# Existem 2 valores ausentes em "Embarked" ( Remover )

Embarked    0
Pclass      0
Sex         0
dtype: int64

#### ``OneHotEncoder`` para codificar as variáveis.

In [16]:
# Gerando objeto One-Hot-Encoder
ohe = OneHotEncoder(sparse_output=False)

# Categorizando as variaveis qualitativas 
X_train_titanic_onehot = ohe.fit_transform(X_train_titanic_)

X_train_titanic_onehot

array([[0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 1., ..., 1., 0., 1.]])

**sparse_output**: Retorna uma matriz esparsa no formato “Compressed Sparse Row” (CSR). Uma matriz é dita esparsa quando possui uma grande quantidade de elementos com valor zero (ou não presentes, ou não necessários). Visualização gráfica de uma matriz esparsa, em que os elementos iguais a zero são representados em branco, e aqueles diferentes de zero em preto.

In [17]:
# Convertendo em um DataFrame as variaveis qualitativas codificadas 
pd.DataFrame(X_train_titanic_onehot, columns=ohe.get_feature_names_out())

,Embarked_C,Embarked_Q,Embarked_S,Pclass_1st,Pclass_2nd,Pclass_3rd,Sex_female,Sex_male
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...
600,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
601,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
602,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
603,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


**Desvantagens:**

* Se a variável possui muitos níveis (estados do brasil por exemplo), muitas colunas serão criadas e isso pode ser um problema em termos de processamento e de alta dimensionalidade (maldição da dimensionalidade).

* Alguns modelos (árvores) se beneficiam mais de outros métodos.

## ``OrdinalEncoder``

É um transformador de variaveis qualitativas em categoricas. Quando houverem variáveis categóricas ordinais, estas serão melhor codificadas pelo OrdinalEncoder. O OrdinalEncoder atribui um número para cada nível da variável, de acordo com a ordem pré-estabelecida pelo usuário.

Preservando a Ordem em Variáveis Categóricas ao contrário do One-Hot Encoder, o Ordinal Encoder é usado quando existe uma ordem natural entre as categorias. Ele atribui valores inteiros sequenciais às categorias, preservando a relação de ordem. Essa abordagem é útil para algoritmos que podem se beneficiar dessa informação, como modelos lineares ou árvores de decisão.


#### Vantagens:

* Preserva a ordem das categorias e atribui valores numéricos com base na ordem especificada.
* Reduz a dimensionalidade do espaço de recursos em comparação com o One-Hot Encoding.


#### Desvantagens:

* Atribui valores numéricos arbitrários às categorias, o que pode introduzir um viés indesejado.
* Algoritmos de aprendizado de máquina podem erroneamente assumir uma relação contínua entre os valores atribuídos.

#### Preparando os Dados

In [18]:
# Obtendo somente a Variavel Qualitativa Ordinal
X_train_titanic_class = X_train_titanic.loc[:, ['Pclass']]

# Atribuindo a uma variavel python a ordem dos elementos da variavel qualitativa ordinal
order_levels = ['1st', '2nd', '3rd']

#### sklearn.preprocessing.``OrdinalEncoder``

* Codifique recursos categóricos como uma matriz inteira. A entrada para este transformador deve ser uma matriz de números inteiros ou strings, denotando os valores assumidos por recursos categóricos (discretos). Os recursos são convertidos em números inteiros ordinais. Isso resulta em uma única coluna de números inteiros (0 a n_categories - 1) por recurso. Retorna um objeto próprio Codificado montado.

**Nota**: Se não especificarmos a ordem dos níveis, o transformador atribui os labels por ordem alfabética nos níveis.

In [19]:
# Gerando um objeto transformador
oe = OrdinalEncoder(categories=[order_levels])


X_train_titanic_oe = oe.fit_transform(X_train_titanic_class)

In [20]:
pd.DataFrame(X_train_titanic_oe, columns=oe.get_feature_names_out()).head()

,Pclass
0,2.0
1,0.0
2,1.0
3,1.0
4,2.0


## ``Frequency Encoder``

A **codificação de frequência**, geralmente conhecida como **codificação de contagem**, é um método comum empregado no **tratamento de dados categóricos** no aprendizado de máquina. Na **codificação de frequência**, **substituímos** cada **categoria** pela **contagem da frequência** com que ela aparece no **conjunto de dados**. Em outras palavras, **substituímos a categoria pela sua frequência**.

Quando temos **muitos níveis nas variáveis categóricas** ou mesmo elas **não possuem nenhum grau ordinal**, podemos utilizar o **Frequency Encoder**. Esse método **substitui cada nível de classe** por sua **frequência nos dados de treino**.

Atenção a diferença entre **codificação de destin** e **codificação de frequência**, em resumo, a **codificação de destino substitui cada categoria pelo valor alvo médio**, enquanto a **codificação de frequência substitui cada categoria por sua frequência de ocorrência** no conjunto de dados .

#### feature_engine.encoding.``CountFrequencyEncoder``

* CountFrequencyEncoder() codificará apenas variáveis ​​categóricas por padrão (tipo 'objeto' ou 'categórico'). Você pode passar uma lista de variáveis ​​para codificar. Alternativamente, o codificador encontrará e codificará todas as variáveis ​​​​categóricas (tipo 'objeto' ou 'categórica'). Este substitui categorias pela contagem ou pela porcentagem de observações por categoria. Por exemplo, na variável cor, se 10 observações forem azuis, o azul será substituído por 10. Alternativamente, se 10% das observações forem azuis, o azul será substituído por 0,1.


In [21]:
# Gerando um objeto transformador utilizando metodo de frequancia aplicado as variaveis 'Embarked', 'Pclass', 'Sex'
cfce = CountFrequencyEncoder(encoding_method='frequency', variables=['Embarked', 'Pclass', 'Sex'])

# Obtendo a categorização das frequancias
X_train_titanic_cfce = cfce.fit_transform(X_train_titanic)

In [22]:
# Gerando um DataFrame com os dados transformados
pd.DataFrame(X_train_titanic_cfce, columns=cfce.get_feature_names_out()).head()

,Embarked,Age,Sex,Pclass
173,0.770248,21.0,0.642975,0.485950
6,0.770248,54.0,0.642975,0.266116
340,0.770248,2.0,0.642975,0.247934
314,0.770248,43.0,0.642975,0.247934
565,0.770248,24.0,0.642975,0.485950


## ``Target Encoder``

O **Codificador de destino**  possui como principal função **codificar as categorias presentes nas variaveis qualitativas e substituindo-as** por uma **medição do efeito** que podem ter no **alvo**, a **variavel dependente, target**.

**Target Encoder** ou **Codificador de destino** é uma técnica de **pré-processamento de dados** que **transforma variáveis categóricas** utilizando a informação do **alvo (target)** associado a **cada categoria**. 

Ao vez de atribuir um único número para cada categoria, o **TargetEncoder substitui cada categoria pelo valor médio** (ou outra estatística) do **alvo** para aquela **categoria específica**.

A ideia básica do **TargetEncoder** é **utilizar a informação do alvo para capturar padrões nos dados** e torná-los mais informativos para os algoritmos de aprendizado de máquina. Isso pode ajudar a **melhorar a performance do modelo**, especialmente quando a **variável categórica tem relação com o alvo**.

#### Vantagens:

* **Incorporação de informações do alvo**: O TargetEncoder utiliza a informação do alvo (target) para criar uma representação numérica das categorias. Isso pode ajudar a capturar relações entre as categorias e o alvo, potencialmente melhorando a performance do modelo.

* **Menor dimensionalidade**: Ao contrário do One-Hot Encoding, o TargetEncoder não aumenta a dimensionalidade dos dados, já que substitui as categorias por valores numéricos. Isso pode ser vantajoso em termos de eficiência computacional e redução da complexidade do modelo.

#### Desvantagens:

* **Sensibilidade a dados desbalanceados**: O TargetEncoder pode ser sensível a dados desbalanceados, principalmente quando a variável categórica tem categorias com poucas observações. Nesses casos, a média das categorias pode não ser uma estimativa confiável, levando a resultados enviesados.

* **Risco de overfitting**: O TargetEncoder pode introduzir informações do alvo diretamente nos dados, o que pode aumentar o risco de overfitting.Se houver categorias raras ou únicas no conjunto de treinamento que não estão presentes no conjunto de teste, o modelo pode ser incapaz de generalizar adequadamente.

* **Problemas com novas categorias**: Assim como o LabelEncoder, o TargetEncoder também pode enfrentar problemas quando novas categorias aparecem nos dados de teste ou produção. Se uma nova categoria não vista durante o treinamento for encontrada, pode ser necessário adotar uma estratégia para lidar com essa situação, como substituir a nova categoria por um valor padrão ou considerar técnicas alternativas de codificação.

In [23]:
# Gerando Objeto transformador aplicado as variaveis 'Embarked', 'Pclass', 'Sex'
te = TargetEncoder(cols=['Embarked', 'Pclass', 'Sex'])

# Obtendo a categorização de acordo com o target
df_te = te.fit_transform(X_train_titanic, y_train_titanic)

# Visualizando a trasformação
display(df_te)

,Embarked,Age,Sex,Pclass
173,0.360515,21.0,0.195373,0.231293
6,0.360515,54.0,0.195373,0.627329
340,0.360515,2.0,0.195373,0.473333
314,0.360515,43.0,0.195373,0.473333
565,0.360515,24.0,0.195373,0.231293
...,...,...,...,...
125,0.566356,12.0,0.195373,0.231293
401,0.360515,26.0,0.195373,0.231293
480,0.360515,9.0,0.195373,0.231293
456,0.360515,65.0,0.195373,0.627329


## ``LabelEncoder``

Label Encoder: É uma técnica simples e eficiente que mapeia cada categoria para um valor numérico único. Ele é frequentemente usado em tarefas de classificação binária ou regressão, onde o objetivo é transformar as categorias em valores que possam ser facilmente processados pelos algoritmos de machine learning.Label Encoder não leva em consideração a ordem ou a relação entre as categorias, o que pode ser um desafio para alguns modelos

Vantagens:
Preserva a ordem das categorias, se houver uma relação ordinal.
Reduz a dimensionalidade do espaço de recursos em comparação com o One-Hot Encoding

Desvantagens:
Pode introduzir uma ordem indesejada se a variável categórica não tiver uma relação ordinal real.
Algoritmos de aprendizado de máquina podem erroneamente assumir uma relação de ordem entre as categorias.